In [6]:
import pandas as pd

def read_sequences_from_file(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequences.append(line.strip())
    return sequences

def find_first_sequence_with_gataa(sequences):
    for sequence in sequences:
        if 'CAATAAAA' in sequence:
            return sequence
    return None

def generate_substrings(base_str):
    substrings = []
    length = len(base_str)
    for i in range(length):
        for j in range(i+1, length+1):
            if base_str[i:j] and len(base_str[i:j]) != length:
                substrings.append(base_str[i:j])
    return sorted(set(substrings), key=len, reverse=True)

def align_sequences(sequences, reference_sequence):
    ref_index = reference_sequence.find('CAATAAAA')
    substrings = generate_substrings('CAATAAAA')

    columns = [i for i in range(-7, 15)]  
    df = pd.DataFrame(columns=columns)

    for sequence in sequences:
        aligned = False
        if 'CAATAAAA' in sequence:
            idx = sequence.find('CAATAAAA')
            aligned = True
            entire = True
        else:
            for substring in substrings:
                if substring in sequence:
                    idx = sequence.find(substring)
                    ref_index2 = reference_sequence.find(substring)
                    entire = False
                    aligned = True
                    break

        if aligned and entire:
            offset = ref_index - idx
            indices = range(offset, offset + len(sequence))
            df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
            
        if aligned and not entire:
            offset = ref_index2 - idx
            indices = range(offset, offset + len(sequence))
            df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
        

    return df

input_file_path = '/Users/bristi/Desktop/Design Project/Working-with-TF/HOXD13_I297V/bin_10/bin_10.fasta'
output_file_path = 'aligned_sequences2.csv'

sequences = read_sequences_from_file(input_file_path)

reference_sequence = find_first_sequence_with_gataa(sequences)
if not reference_sequence:
    raise ValueError("No sequence containing 'CAATAAAA' was found in the file.")

df_aligned = align_sequences(sequences, reference_sequence)

/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_1063/1808640233.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_1063/1808640233.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_1063/1808640233.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_1063/1808640233.py:56: FutureWarning: The frame.append method 

In [7]:
df_aligned

,-7,-6,-5,-4,-3,-2,-1,0,1,2,...,7,8,9,10,11,12,13,14,-9,-8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,A,A,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,C,.,G,C,T,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,A,.,G,G,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,...,G,A,.,.,C,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A,G,T,.,.,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,A,...,A,T,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,C,A,...,A,C,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,.,G,.,A,G,A,NaN,NaN,NaN,NaN
9970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,.,...,A,G,T,T,NaN,NaN,NaN,NaN,NaN,NaN


# Generating txt file for weblogo

In [8]:
columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
extracted_data = df_aligned[columns_to_extract]
extracted_data = extracted_data.fillna('-')

formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

output_file_path = 'HOXD_13_BIN10_aligned.txt'
with open(output_file_path, 'w') as file:
    for line in formatted_data:
        file.write(line + '\n')